In [ ]:
#Download required imports
import sys
!{sys.executable} -m pip install torch
!{sys.executable} -m pip install torchcodec
!{sys.executable} -m pip install evaluate
!{sys.executable} -m pip install jiwer
!{sys.executable} -m pip install transformers[torch]


In [10]:
#Import dataset and split
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset("rishabbahal/quebecois_canadian_french_dataset", "default", split="train")
common_voice["test"] = load_dataset("rishabbahal/quebecois_canadian_french_dataset", "default", split="test")

print(common_voice)

Generating test split: 100%|██████████| 1348/1348 [00:00<00:00, 6838.85 examples/s]


DatasetDict({
    train: Dataset({
        features: ['audio', 'text', 'audio_filepath', '__index_level_0__'],
        num_rows: 5389
    })
    test: Dataset({
        features: ['audio', 'text', 'audio_filepath', '__index_level_0__'],
        num_rows: 1348
    })
})


In [11]:
#Keep only needed data (audio and text)
common_voice = common_voice.remove_columns(["audio_filepath", "__index_level_0__"])

print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['audio', 'text'],
        num_rows: 5389
    })
    test: Dataset({
        features: ['audio', 'text'],
        num_rows: 1348
    })
})


In [12]:
#Load feature extractor from pre-trained check-point
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

In [13]:
#Load tokenizer
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="French", task="transcribe")

In [14]:
#Combine feature extractor and tokenizer to create the processor
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="French", task="transcribe")

In [15]:
#Prepare data
sample = common_voice["train"][2]
audio_decoder = sample['audio']
audio_samples = audio_decoder.get_all_samples()

audio_array = audio_samples.data.squeeze()
sampling_rate = audio_samples.sample_rate

print(f"Audio array shape: {audio_array.shape}")
print(f"Sampling rate: {sampling_rate}")

Audio array shape: torch.Size([36784])
Sampling rate: 16000


In [16]:
#Prepare dataset
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]
    audio_samples = audio.get_all_samples()
    audio_array = audio_samples.data.squeeze(0).numpy()

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio_array, sampling_rate=16000).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["text"]).input_ids
    return batch

In [17]:
#Apply data preparation function to training examples
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=1)

Map (num_proc=1): 100%|██████████| 1348/1348 [00:20<00:00, 67.21 examples/s]


In [18]:
#Load pre-trained checkpoint
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

In [19]:
#Disable automatic language detection and force model to generate french
model.generation_config.language = "french"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None

In [20]:
#Define data collector
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [21]:
#Initialize data collector
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [22]:
#Choose evaluation metrics (error rate)
import evaluate

metric = evaluate.load("wer")

In [23]:
#Define a function that takes the model's predictions and returns the evaluation metric
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [24]:
import shutil
import os

ckpt_dir = "/scratch/lemun9@ulaval.ca/whisper_checkpoints"

if os.path.exists(ckpt_dir):
    shutil.rmtree(ckpt_dir)

In [31]:
#Define training configuration
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir = "/scratch/lemun9@ulaval.ca/whisper_checkpoints",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    eval_strategy="steps", 
    save_strategy="steps", 
    save_steps=2000,
    eval_steps=2000, 
    gradient_checkpointing=False,
    fp16=True,
    per_device_eval_batch_size=1,
    predict_with_generate=True,
    generation_max_length=225,
    logging_steps=25,
    report_to=[],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
    save_only_model=True,
    save_total_limit=2,
)

In [32]:
#Forward training agruments, model, dataset, data collector, and compute metrics function to HuggingFace trainer
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    processing_class=processor
)

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [33]:
#Save processor object
processor.save_pretrained(training_args.output_dir)

[]

In [34]:
#Make space for the model training
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [35]:
#Train the model
trainer.train()

Step,Training Loss,Validation Loss,Wer
2000,0.417200,0.849176,52.090543
4000,0.071600,1.051465,52.374199


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/home/ulaval.ca/lemun9/env_hf/lib/python3.12/site-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}. You are seeing this

TrainOutput(global_step=4000, training_loss=0.4412349570989609, metrics={'train_runtime': 7736.5838, 'train_samples_per_second': 4.136, 'train_steps_per_second': 0.517, 'total_flos': 9.2304040292352e+18, 'train_loss': 0.4412349570989609, 'epoch': 5.93523844869178})

In [2]:
#Load trained model
from transformers import WhisperForConditionalGeneration, WhisperProcessor
import torch
import soundfile as sf

processor = WhisperProcessor.from_pretrained("/scratch/lemun9@ulaval.ca/whisper_checkpoints")

model = WhisperForConditionalGeneration.from_pretrained("/scratch/lemun9@ulaval.ca/whisper_checkpoints/checkpoint-4000")

model.eval()

/home/ulaval.ca/lemun9/env_hf/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 768, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(768, 768, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 768)
      (layers): ModuleList(
        (0-11): 12 x WhisperEncoderLayer(
          (self_attn): WhisperAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (f

In [3]:
def transcribe_from_audio_with(model, processor, audio_array):
    inputs = processor(audio_array, sampling_rate=16000, return_tensors="pt")

    with torch.no_grad():
        ids = model.generate(inputs["input_features"], max_length=225)

    txt = processor.tokenizer.batch_decode(ids, skip_special_tokens=True)[0]
    return txt

In [3]:
#Compare with test data
from datasets import load_dataset, DatasetDict

common_voice_raw = DatasetDict()
common_voice_raw["train"] = load_dataset("rishabbahal/quebecois_canadian_french_dataset", split="train")
common_voice_raw["test"] = load_dataset("rishabbahal/quebecois_canadian_french_dataset", split="test")

sample = common_voice_raw["test"][1]
audio = sample["audio"]["array"]
sr = sample["audio"]["sampling_rate"]
expected_text = sample["text"]
print("Reference text :", expected_text)

txt_ft = transcribe_from_audio_with(model, processor, audio)

print("Fine-tuned :", txt_ft)

import evaluate
metric = evaluate.load("wer")

wer_ft = metric.compute(predictions=[txt_ft], references=[expected_text])

print("WER fine tuned:", wer_ft)


ModuleNotFoundError: No module named 'pyarrow'

In [4]:
from jiwer import wer
from tqdm import tqdm
import torch

# Make sure models are on GPU
model = model.to("cuda")

def compute_dataset_wer(model, processor, dataset):
    preds = []
    refs = []
    for item in tqdm(dataset, desc="Processing"):
        audio = item["audio"]["array"]
        text = item["text"]
        inputs = processor(audio, sampling_rate=16000, return_tensors="pt")
        with torch.no_grad():
            ids = model.generate(inputs["input_features"].to("cuda"), max_length=225)
        pred = processor.tokenizer.batch_decode(ids, skip_special_tokens=True)[0]
        preds.append(pred)
        refs.append(text)
    return wer(refs, preds)

# Use the RAW dataset
from datasets import load_dataset

common_voice_raw = load_dataset("rishabbahal/quebecois_canadian_french_dataset", split="test")

print("Evaluating fine-tuned model...")
wer_finetuned = compute_dataset_wer(model, processor, common_voice_raw)


print(f"\n{'='*60}")
print("RESULTS")
print(f"{'='*60}")
print(f"WER fine-tuned: {wer_finetuned*100:.2f}%")

Evaluating fine-tuned model...


Processing:   0%|          | 0/1348 [00:00<?, ?it/s]`generation_config` default values have been modified to match model-specific defaults: {'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}. If this is not desired, please set these values explicitly.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
A custom logits pro


RESULTS
WER fine-tuned: 52.34%
